In [5]:
#Elis Island
#Here we are just Importing the necessary modules and
import variables_cartpole as exp
import numpy as np
import scipy as sci
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from pydmd import DMD
from pydmd import DMDc
import matplotlib.animation as animation
import HBDMDC as mydmd
import myfunctions as myfun

#Plotting configuration
np.set_printoptions(precision=5, suppress=True)
plt.style.use('bmh')
figsize = (12, 10)
figsize2 = (12, 5)
dpi = 600    # Resolution in dots per inch

# System measurements
m = exp.m
M = exp.M
g = exp.g
l = exp.l
t_span = exp.t_span
dt = exp.dt
b = exp.b
K = exp.K
B = exp.B
#B = np.array([[0],[1/(m*l**2)]])
nvar = int(exp.nvari)
polyorder = int(exp.order)
sineorder = int(exp.sine)
duration = 30

Above we just imputed static characteristics for the system and the linearized SS model.
The LQR derived gain is also imported as well as some of the lifting characteristics. In this specific
example we are preparing to lift the state measurements using a {{$polyorder$}}
order polynomial approximation and a {{$sineorder$}} Fourier approximation.

In [11]:
#Initial Conditions and non-linear dynamics

x0 = np.array([-1, 0.0,np.pi+.0001,0.])
xf = np.array([1.0,0.,np.pi,0.])
def f(t,xk):
    Sy = np.sin(xk[2])
    Cy = np.cos(xk[2])
    D = m * l*l*(M+m * (1.0-Cy**2))
    x_1 = xk[1]
    x_2 = 1.0/D * (-m**2*l**2*g*Cy*Sy + (m*l**2)*(m*l*Sy*xk[3]**2))
    u1 =  m*l*l*(1/D)*u(xk)
    x_3 = xk[3]
    x_4 = (1.0/D)*((m+M)*m*g*l*Sy - m*l*Cy*(m*l*Sy*xk[3]**2))
    u2 = -m*l*Cy*(1.0/D)*u(xk)
    return [x_1, x_2+u1,x_3, x_4+u2]
#from control.matlab import *
import control
Kg, S, E = lqr(exp.A, exp.B, exp.Q, exp.R)
print(Kg)

ControlSlycot: can't find slycot module 'sb02md' or 'sb02nt'

Setup of initial & terminal conditions and definition of nonlinear dynamics for the pendulum system.
Below we run an uncontrolled simulation with the initial conditions {{x0}}. This pendulum setup is undamped and should show a
cyclic behavior which we will then use to feed the DMD algorithms and find the respective Koopman Operators. We will (probably)
not use this specific data set to generate control laws for the system but we will use it as a sanity check for the fidelity
of DMD constructed systems.
u = lambda x: 0

In [ ]:
u = lambda x: 0
#Uncontrol Simulation
y0 = solve_ivp(f,[0.0,duration],x0,method='RK45',t_eval=t_span)
#figure

myfun.showstate(y0.y, t_span,'Uncontrolled Pendulum', figsize, dpi)

Below we generate a controlled version of the above system. In this example we will be bringing the system to a terminal state of
{{xf}}. This data will be fed through the DMD algorithms and eventually used to generate learned dynamics later.

The method we will use to control this system initially to learn the Koopman dynamics is through a Linear Quadratic Regulator [(LQR)](https://en.wikipedia.org/wiki/Linear%E2%80%93quadratic_regulator)

In [ ]:
#Control Implementation
u = lambda x: -np.matmul(K, (x - xf))
y1 = solve_ivp(f,[0.0,duration],x0,method='RK45',t_eval=t_span)

myfun.showstate(y1.y, t_span,'Controlled Pendulum', figsize, dpi)

#recording control input

findu = y1.y
u = np.zeros(int(len(t_span)-1))
unull = np.zeros(int(len(t_span)-1))
for o in range(int(len(t_span)-1)):
    ynow = findu[:,o]
    u[o] = -np.matmul(K,(ynow-xf))
u.resize((1,int(len(t_span)-1)),refcheck=False)
unull.resize((1,int(len(t_span)-1)),refcheck=False)
plt.figure(figsize=figsize2, dpi=dpi)
plt.title("LQR Generated Control Signal")
plt.plot(u[0,:])
plt.xlabel(r"$t$ [ms]")
plt.ylabel(r"u")
plt.show()

The control is implimented via the function $u = -K(x-x_f)$ where $u$ is the control signal, $x$ is the current state, $x_f$ is the desired state and $K$ is the
gain found through LQR. The above figure is the control signal following the LQR control law imposed on the pendulum system.

Below we will use the [DMD](https://mathlab.github.io/PyDMD/index.html) algorithm to find the Koopman operator of the
uncontrolled system using the data from the non-linear model. For this run we will not be truncating the data.
<details>
<summary> DMD Algorithm </summary>
    Starting with the systems $x_{k+1} = Ax_k$ and a set of 2 consecutive snap shots $X$ and $X'$ of the format

    1. Take singular value decomposition (SVD) of $X$ <br>
        $$ \bf{\mathbf{X}} \approx U \Sigma V^* $$
        where $*$ denotees the conjugate transpose, $U \in \mathbb{R}^{n\times r}$, $\Sigma \in \mathbb{R}^{r\times r}$ and $V \in \mathbb{R}^{n\times r}$
        $r$ is the rank of the reduced SVD approximation to $X$. In SVD, the columns of $U$ are orthonormal and are POD Modes.
        > Some notes on SVD: The SVD reduction in this equation is exploited at this stage in the algorithm to perform a low-rank truncation of the data; if within the data there is a low-dimensional structure the singular values of $\Sigma$ will rapidly decrease to zero with a finite low number of dominant nodes
    2. The matrix $A$ is then found by the pseudoinverse of $X$ obtained via the SVD:
        $$ A\approx \Bar{A} =& \bf{\mathbf{X}}'\tilde{V}\tilde{\Sigma}^{-1}\tilde{U}^* $$
        Here the matrix $A\approx$ defines a "low-dimensional" linear model of the dynamical system on the POD coordinates:
        $$\tilde{\mathbf{x}}_{k+1} =& \tilde{A}\tilde{\mathbf{x}}_k $$
    3. Now perform the eigendecomposition of $A\approx$:
        $$ AW = W\mathbf{\Lambda} $$
        Here the columns of $W$ are the eigenvectors and $\Lambda$ is a diagonal matrix containing the corresponding eigenvalues
    4. This step is not so necessary to this application but for completion it will be included. To reconstruct the eigendecomposition of $A$ we will use $W$ and $\Lambda$. Since we have the eigenvalues of $A$ to find the corresponding eigenvectors of $A$ (these are the DMD modes!!) we use the folloing equation for $Phi$:
        $$\math{\Phi} =& \math{X}'\mathbf{V\Sigma}^{-1}\mathbf{W}$$


</details>

In [ ]:
#print(t_span)
ybase = y0.y
[A1, Phi1, Xdmd1] = mydmd.mydmd(ybase, 4, dt)

[y2] = myfun.buildlinsys(A1,B, ybase[:,0], unull, t_span)
#y2 = Y2.resize((len(ybase), len(ybase[1])), refcheck=False)
# y2test= np.real(Xdmd)
myfun.comparestates(ybase,Xdmd1,t_span,'Testing','Original','Reconstructed',figsize, dpi)

Something is clearly wrong with state 2 despite correct A matrix, investigation is in order

Now going to perform regular dmd on controlled data

In [ ]:
ybase = y1.y
[A2, Phi2, Xdmd2] = mydmd.mydmd(ybase, 4, dt)

[y3] = myfun.buildlinsys(A2,B, ybase[:,0], unull, t_span)

myfun.comparestates(ybase,Xdmd2,t_span,'Controlled Pendulum States','Original','Reconstructed',figsize, dpi)

Now to DMDC

In [ ]:
ybase = y1.y
[A3, B3, Phi3, Xdmd3] = mydmd.myDMDcUB(ybase, u, 4, 1, dt)
[y4] = myfun.buildlinsys(A3,B3, ybase[:,0], u, t_span)

#y4.resize((len(ybase), len(ybase[1])), refcheck=False)
myfun.comparestates(ybase,y4,t_span,'Testing','Original','Reconstructed',figsize,dpi)

We can see the reconstructed state diverging rapidly. This is similar to my MATLAB result but the scaling is quite off

Now I will move on to EDMDc

In [ ]:
indata = y0.y
y3 = myfun.pool(indata.T,nvar,polyorder,0)
q = np.size(y3,1)
[A4, Phi4, Xdmd4] = mydmd.mydmd(y3.T, q, dt)

In [ ]:
myfun.comparestates(y3.T,Xdmd4,t_span,'Testing','Original','Reconstructed',figsize,dpi)

Now performing Edmd on controlled data

In [ ]:
indata = y1.y
y4 = myfun.pool(indata.T,nvar,polyorder,0)
q = np.size(y4,1)

[A5, Phi5, Xdmd5] = mydmd.mydmd(y4.T, q, dt)
myfun.comparestates(y4.T,Xdmd5,t_span,'Testing','Original','Reconstructed',figsize,dpi)

The plot looks like th ereconstructed time is 1 step ahead of the real data but that could just be a response to the slightly smaller
data set

EDMDc Time
Edmdc using zeros!!!

In [ ]:
nulctrl = np.zeros((np.size(u,0),np.size(u,1)))
indata = y0.y
y5 = myfun.pool(indata.T,nvar,polyorder,0)
q = np.size(y5,1)

[A6,B6,Phi6, Xdmd6] = mydmd.myDMDcUB(y5.T,nulctrl,q,1,dt)
# [A3, B3, Phi3, Xdmd3] = mydmd.myDMDcUB(ybase, u, 2, 1, dt)
myfun.comparestates(y5.T,Xdmd6,t_span,'Testing','Original','Reconstructed',figsize,dpi)

In [ ]:
Y0 = y5.T[:,0]
# print(np.size(Y0))
# print((np.size(y5,1)))
# Y0.resize((np.size(y5,1),1),refcheck=False)
Y5 = myfun.buildlinsys(A6,B6, Y0,nulctrl,t_span)
n = np.size(A6,1)
m = np.size(t_span,0)
Y = np.zeros((n,m-1))
Y1 = np.zeros((n,m-1),dtype=complex)
Y[:, 0] = Y0
Y1[:,0] = Xdmd6[:,0]
for i in range(0, m - 2):
    Y[:, i + 1] = A6 @ Y[:, i] + B6 @ nulctrl[:, i]
    Y1[:,i+1] = Xdmd6[:,i] + B6 @ nulctrl[:,i]
# myfun.showstate(Y,t_span,'Building EDMDc',figsize,dpi)
myfun.comparestates(Y1,Y,t_span,'EDMDc on 0 Ctrl','XDMD','Ax + Bu',figsize,dpi)

EDMDc with of controlled data

In [ ]:
ndata = y1.y
y6 = myfun.pool(indata.T,nvar,polyorder,0)
q = np.size(y6,1)

[A7,B7,Phi7, Xdmd7] = mydmd.myDMDcUB(y6.T,u,q,1,dt)
# [A3, B3, Phi3, Xdmd3] = mydmd.myDMDcUB(ybase, u, 2, 1, dt)
myfun.comparestates(y6.T,Xdmd7,t_span,'Testing','Original','Reconstructed',figsize,dpi)

In [ ]:
Y0 = y6.T[:,0]
# print(np.size(Y0))
# print((np.size(y5,1)))
# Y0.resize((np.size(y5,1),1),refcheck=False)
# Y5 = myfun.buildlinsys(A6,B6, Y0,nulctrl,t_span)
n = np.size(A7,1)
m = np.size(t_span,0)
Y3 = np.zeros((n,m-1))
Y4 = np.zeros((n,m-1),dtype=complex)
Y3[:, 0] = Y0
Y4[:,0] = Xdmd6[:,0]
for i in range(0, m - 2):
    Y3[:, i + 1] = A7 @ Y3[:, i] + B7 @ u[:, i]
    Y4[:,i+1] = Xdmd7[:,i] + B7 @ u[:,i]
# myfun.showstate(Y,t_span,'Building EDMDc',figsize,dpi)
myfun.comparestates(Y4,Y3,t_span,'EDMDc on lqr Ctrl','XDMD','Ax + Bu',figsize,dpi)
myfun.comparestates(y6.T,Y4,t_span,'EDMDc on lqr Ctrl','Lifted Base State','XDMD + Bu',figsize,dpi)